In [1]:
# import the necessary packages
from skimage.segmentation import slic
from skimage.segmentation import mark_boundaries
from skimage.util import img_as_float
from skimage import io
import matplotlib.pyplot as plt
import argparse
import datetime
import os
import pymysql
import numpy as np
import math
import cv2
from os import listdir
import matplotlib.pyplot as plt
from cmocean import cm
from collections import namedtuple
import stepconvert as sc
import shutil
import seaborn as sns
from copy import deepcopy
import scipy.signal
import matplotlib.pylab as pylab
import pandas as pd
import json
% matplotlib inline
NUM_PICTURES = 64

import graphviz
from multiprocessing import Pool, Lock, Queue, Pipe, Process
import sys
NUM_PROCESSES = 12

In [2]:
theta_l = 65 * math.pi / 180
f = 1 / math.tan(theta_l / 2)
f_pix = 360 * f # the 360 is pixels, half img width, not degrees




theta_r = lambda: ((513 - 1) / NUM_PICTURES) * 2 * math.pi / 513
delta_col = lambda: 2*f_pix*math.sin(theta_r()/2)


s = f_pix
x2xp = lambda x: s * math.atan(x / f_pix)
xp2x = lambda xp: f_pix * math.tan(xp / s)
y2yp = lambda x, y: s * y / math.sqrt((x**2) + (f_pix**2))
yp2y = lambda xp, yp: f_pix * yp * (1/math.cos(xp / s)) / s

In [8]:
def parse_ir(f_name):
    out = []
    vals = []
    # print(f_name)
    with open(f_name) as FILE:
        vals = [float(n) for n in FILE.read().strip().split(',')]
    for i in range(0, 64, 4):
        out.append(vals[i:i+4])
    out = np.array(out)
    #
    out = np.flipud(out)
    out = np.fliplr(out)

    return out

In [4]:
def ir_image(im, max_temp, min_temp):
    therm_im = np.zeros(list(im.shape)+[3],dtype=np.uint8)
    mid_temp = (max_temp + min_temp)/2
    rng_temp = max_temp - min_temp
    #[H, S, V]
    therm_im[:,:,0] = (170 * (max_temp - im) / rng_temp) 
    therm_im[:,:,1] = 255
    therm_im[:,:,2] = 127
    
    # therm_im = cv2.cvtColor(therm_im, cv2.COLOR_HSV2BGR)
    # therm_im = ,   cmap=cm.thermal ,vmin = 65, vmax = 100)
    return cv2.cvtColor(therm_im, cv2.COLOR_HSV2BGR)

In [5]:
def gs_image(im, max_temp, min_temp):
    rng_temp = max_temp-min_temp
    therm_im = np.zeros(im.shape, dtype=np.uint8)
    therm_im = 255*(im-min_temp)//rng_temp
    return therm_im

In [6]:
def reconvertTemp(thPan, max_temp, min_temp):
    return ((max_temp-min_temp) * thPan / 255) + min_temp

In [7]:
def alignment_matrix():
    mats = []
    
    # (x, y)
    src = np.array([
        (0, 0),
        (2, 10),
        (1, 11),
        (3, 5)
    ], dtype = np.float32)
    dst = np.array([
        (171, 713),
        (324, 239),
        (194, 209),
        (380, 445)
    ], dtype = np.float32)
    mats.append(cv2.getPerspectiveTransform(src, dst))
    
    src = np.array([
        (1, 8),
        (2, 4),
        (3, 12),
        (0, 7)
    ], dtype = np.float32)
    dst = np.array([
        (257, 311),
        (340, 490),
        (372, 165),
        (208, 356)
    ], dtype = np.float32)
    mats.append(cv2.getPerspectiveTransform(src, dst))
    
    src = np.array([
        (3, 12),
        (3, 7),
        (0, 4),
        (2, 8)
    ], dtype = np.float32)
    dst = np.array([
        (380, 160),
        (336, 386),
        (100, 500),
        (279, 352)
    ], dtype = np.float32)
    mats.append(cv2.getPerspectiveTransform(src, dst))
    
    return sum(mats)/len(mats)
    

mat = alignment_matrix()

# Move Files to the locations

In [9]:
# directory = '/home/nilavro/data/f03/2017-12-08/1512681434856'
# directory = '/home/nilavro/data/f03/2017-12-15/1513304232411'
# directory = '/home/nilavro/data/f03/2017-12-17/1513361832444'
# directory = '/home/nilavro/data/f03/2017-12-18/1513549032246'
# directory = '/home/nilavro/data/f03/2017-12-19/1513664059443'
# directory = '/home/nilavro/data/f03/2017-12-19/1513685832345'
# directory = '/home/nilavro/data/f03/2017-12-19/1513711032463'
directory = '/home/nilavro/data/Nilavra_z1/2017-12-20/1513736748855'

# directory = '/home/nilavro/data/f03/2017-12-20/1513740898406'

In [10]:

for dir in sorted(listdir(directory)):

    dirname = directory + '/' + dir  + '/'
    
    print(dirname)
    sc.main(dirname)
    # shutil.rmtree(dirname +  'temperatures/ir_folder0') 
    # shutil.rmtree(dirname +  'rotated/rotated_folder0')
    
 
    
 

/home/nilavro/data/Nilavra_z1/2017-12-20/1513736748855/1513746002780/
Starting ...
Read EEPROM data... 
Calculated constants ...
Processing 000 ... Processing 001 ... Processing 002 ... Processing 003 ... Processing 004 ... Processing 005 ... Processing 006 ... Processing 007 ... Processing 008 ... Processing 009 ... Processing 010 ... Processing 011 ... Processing 012 ... Processing 013 ... Processing 014 ... Processing 015 ... Processing 016 ... Processing 017 ... Processing 018 ... Processing 019 ... Processing 020 ... Processing 021 ... Processing 022 ... Processing 023 ... Processing 024 ... Processing 025 ... Processing 026 ... Processing 027 ... Processing 028 ... Processing 029 ... Processing 030 ... Processing 031 ... Processing 032 ... Processing 033 ... Processing 034 ... Processing 035 ... Processing 036 ... Processing 037 ... Processing 038 ... Processing 039 ... Processing 040 ... Processing 041 ... Processing 042 ... Processing 043 ... Processing 044 ... Processing 045 .

Processing 000 ... Processing 001 ... Processing 002 ... Processing 003 ... Processing 004 ... Processing 005 ... Processing 006 ... Processing 007 ... Processing 008 ... Processing 009 ... Processing 010 ... Processing 011 ... Processing 012 ... Processing 013 ... Processing 014 ... Processing 015 ... Processing 016 ... Processing 017 ... Processing 018 ... Processing 019 ... Processing 020 ... Processing 021 ... Processing 022 ... Processing 023 ... Processing 024 ... Processing 025 ... Processing 026 ... Processing 027 ... Processing 028 ... Processing 029 ... Processing 030 ... Processing 031 ... Processing 032 ... Processing 033 ... Processing 034 ... Processing 035 ... Processing 036 ... Processing 037 ... Processing 038 ... Processing 039 ... Processing 040 ... Processing 041 ... Processing 042 ... Processing 043 ... Processing 044 ... Processing 045 ... Processing 046 ... Processing 047 ... Processing 048 ... Processing 049 ... Processing 050 ... Processing 051 ... Processing 0

Processing 000 ... Processing 001 ... Processing 002 ... Processing 003 ... Processing 004 ... Processing 005 ... Processing 006 ... Processing 007 ... Processing 008 ... Processing 009 ... Processing 010 ... Processing 011 ... Processing 012 ... Processing 013 ... Processing 014 ... Processing 015 ... Processing 016 ... Processing 017 ... Processing 018 ... Processing 019 ... Processing 020 ... Processing 021 ... Processing 022 ... Processing 023 ... Processing 024 ... Processing 025 ... Processing 026 ... Processing 027 ... Processing 028 ... Processing 029 ... Processing 030 ... Processing 031 ... Processing 032 ... Processing 033 ... Processing 034 ... Processing 035 ... Processing 036 ... Processing 037 ... Processing 038 ... Processing 039 ... Processing 040 ... Processing 041 ... Processing 042 ... Processing 043 ... Processing 044 ... Processing 045 ... Processing 046 ... Processing 047 ... Processing 048 ... Processing 049 ... Processing 050 ... Processing 051 ... Processing 0

Processing 000 ... Processing 001 ... Processing 002 ... Processing 003 ... Processing 004 ... Processing 005 ... Processing 006 ... Processing 007 ... Processing 008 ... Processing 009 ... Processing 010 ... Processing 011 ... Processing 012 ... Processing 013 ... Processing 014 ... Processing 015 ... Processing 016 ... Processing 017 ... Processing 018 ... Processing 019 ... Processing 020 ... Processing 021 ... Processing 022 ... Processing 023 ... Processing 024 ... Processing 025 ... Processing 026 ... Processing 027 ... Processing 028 ... Processing 029 ... Processing 030 ... Processing 031 ... Processing 032 ... Processing 033 ... Processing 034 ... Processing 035 ... Processing 036 ... Processing 037 ... Processing 038 ... Processing 039 ... Processing 040 ... Processing 041 ... Processing 042 ... Processing 043 ... Processing 044 ... Processing 045 ... Processing 046 ... Processing 047 ... Processing 048 ... Processing 049 ... Processing 050 ... Processing 051 ... Processing 0

Processing 000 ... Processing 001 ... Processing 002 ... Processing 003 ... Processing 004 ... Processing 005 ... Processing 006 ... Processing 007 ... Processing 008 ... Processing 009 ... Processing 010 ... Processing 011 ... Processing 012 ... Processing 013 ... Processing 014 ... Processing 015 ... Processing 016 ... Processing 017 ... Processing 018 ... Processing 019 ... Processing 020 ... Processing 021 ... Processing 022 ... Processing 023 ... Processing 024 ... Processing 025 ... Processing 026 ... Processing 027 ... Processing 028 ... Processing 029 ... Processing 030 ... Processing 031 ... Processing 032 ... Processing 033 ... Processing 034 ... Processing 035 ... Processing 036 ... Processing 037 ... Processing 038 ... Processing 039 ... Processing 040 ... Processing 041 ... Processing 042 ... Processing 043 ... Processing 044 ... Processing 045 ... Processing 046 ... Processing 047 ... Processing 048 ... Processing 049 ... Processing 050 ... Processing 051 ... Processing 0

...

Processing 000 ... Processing 001 ... Processing 002 ... Processing 003 ... Processing 004 ... Processing 005 ... Processing 006 ... Processing 007 ... Processing 008 ... Processing 009 ... Processing 010 ... Processing 011 ... Processing 012 ... Processing 013 ... Processing 014 ... Processing 015 ... Processing 016 ... Processing 017 ... Processing 018 ... Processing 019 ... Processing 020 ... Processing 021 ... Processing 022 ... Processing 023 ... Processing 024 ... Processing 025 ... Processing 026 ... Processing 027 ... Processing 028 ... Processing 029 ... Processing 030 ... Processing 031 ... Processing 032 ... Processing 033 ... Processing 034 ... Processing 035 ... Processing 036 ... Processing 037 ... Processing 038 ... Processing 039 ... Processing 040 ... Processing 041 ... Processing 042 ... Processing 043 ... Processing 044 ... Processing 045 ... Processing 046 ... Processing 047 ... Processing 048 ... Processing 049 ... Processing 050 ... Processing 051 ... Processing 0

Processing 000 ... Processing 001 ... Processing 002 ... Processing 003 ... Processing 004 ... Processing 005 ... Processing 006 ... Processing 007 ... Processing 008 ... Processing 009 ... Processing 010 ... Processing 011 ... Processing 012 ... Processing 013 ... Processing 014 ... Processing 015 ... Processing 016 ... Processing 017 ... Processing 018 ... Processing 019 ... Processing 020 ... Processing 021 ... Processing 022 ... Processing 023 ... Processing 024 ... Processing 025 ... Processing 026 ... Processing 027 ... Processing 028 ... Processing 029 ... Processing 030 ... Processing 031 ... Processing 032 ... Processing 033 ... Processing 034 ... Processing 035 ... Processing 036 ... Processing 037 ... Processing 038 ... Processing 039 ... Processing 040 ... Processing 041 ... Processing 042 ... Processing 043 ... Processing 044 ... Processing 045 ... Processing 046 ... Processing 047 ... Processing 048 ... Processing 049 ... Processing 050 ... Processing 051 ... Processing 0

Processing 000 ... Processing 001 ... Processing 002 ... Processing 003 ... Processing 004 ... Processing 005 ... Processing 006 ... Processing 007 ... Processing 008 ... Processing 009 ... Processing 010 ... Processing 011 ... Processing 012 ... Processing 013 ... Processing 014 ... Processing 015 ... Processing 016 ... Processing 017 ... Processing 018 ... Processing 019 ... Processing 020 ... Processing 021 ... Processing 022 ... Processing 023 ... Processing 024 ... Processing 025 ... Processing 026 ... Processing 027 ... Processing 028 ... Processing 029 ... Processing 030 ... Processing 031 ... Processing 032 ... Processing 033 ... Processing 034 ... Processing 035 ... Processing 036 ... Processing 037 ... Processing 038 ... Processing 039 ... Processing 040 ... Processing 041 ... Processing 042 ... Processing 043 ... Processing 044 ... Processing 045 ... Processing 046 ... Processing 047 ... Processing 048 ... Processing 049 ... Processing 050 ... Processing 051 ... Processing 0

Processing 000 ... Processing 001 ... Processing 002 ... Processing 003 ... Processing 004 ... Processing 005 ... Processing 006 ... Processing 007 ... Processing 008 ... Processing 009 ... Processing 010 ... Processing 011 ... Processing 012 ... Processing 013 ... Processing 014 ... Processing 015 ... Processing 016 ... Processing 017 ... Processing 018 ... Processing 019 ... Processing 020 ... Processing 021 ... Processing 022 ... Processing 023 ... Processing 024 ... Processing 025 ... Processing 026 ... Processing 027 ... Processing 028 ... Processing 029 ... Processing 030 ... Processing 031 ... Processing 032 ... Processing 033 ... Processing 034 ... Processing 035 ... Processing 036 ... Processing 037 ... Processing 038 ... Processing 039 ... Processing 040 ... Processing 041 ... Processing 042 ... Processing 043 ... Processing 044 ... Processing 045 ... Processing 046 ... Processing 047 ... Processing 048 ... Processing 049 ... Processing 050 ... Processing 051 ... Processing 0

FileNotFoundError: [Errno 2] No such file or directory: '/home/nilavro/data/Nilavra_z1/2017-12-20/1513736748855/1513769402936/eeprom.csv'

# Panorama

In [ ]:
def process_set(setname):
    # Define subdirectories
    IRname  = setname + 'temperatures/'
    dirname = setname + 'rotated/'
    
    
    # Read in the IR images
    # print( sorted(listdir(IRname)))
    
    
    # print(IRname)
    try:
        th_ims = [parse_ir(IRname + im) for im in sorted(listdir(IRname))]
        
    except FileNotFoundError:
        # print(IRname + im)
        return
    # find the min and max temperatures
    max_temp = np.amax(th_ims)
    min_temp = np.amin(th_ims)
    # Create false-color copies of the IR images
    ir_ims = [ir_image(im, max_temp, min_temp) for im in th_ims]
    # Create grey-scaled copies of the IR images
    th_ims = [gs_image(im, max_temp, min_temp) for im in th_ims]
    # Warp the thermal images on to the spaces of the RGB images
    w_ims = [cv2.warpPerspective(im, mat, (480, 720)) for im in ir_ims]
    w_ths = [cv2.warpPerspective(im, mat, (480, 720)) for im in th_ims]
    # Read the RGB images
    ims = [ cv2.flip(cv2.imread(dirname + img),  0  ) for img in sorted(listdir(dirname)) if img[-4:] == '.png']
    # Create single-image RGB/IR overlays
    alpha = 0.5
    
    try:
        o_ims = [cv2.addWeighted(ims[i],alpha,w_ims[i],1-alpha,0) for i in range(NUM_PICTURES)]
    except IndexError:
        #print("not enough images", len(ims), len(w_ims))
        return
    # Do cylindrical warping
    cyl_ims = []
    cyl_ths = []
    (lb,rb) = (13,467)
    
    
    
    center = (ims[0].shape[1]/2, ims[0].shape[0]/2)
    mat_x = np.zeros(ims[0].shape[:2], np.float32)
    mat_y = np.zeros(ims[0].shape[:2], np.float32)
    
    
    for row in range(ims[0].shape[0]):
        for col in range(ims[0].shape[1]):
            x = col - center[0]
            y = row - center[1]
            mat_y[row, col] = yp2y(x, y) + center[1]
            mat_x[row, col] = xp2x(x) + center[0]

    # Trim down all the warped images to rectangles
    cyl_vis = [cv2.remap(im, mat_x, mat_y, cv2.INTER_LINEAR)[145:,195:377] for i,im in enumerate(ims)]
    for i,im in enumerate(o_ims):
        #reim = cv2.remap(im, mat_x, mat_y, cv2.INTER_LINEAR)[:,lb:rb]
        reim = cv2.remap(im, mat_x, mat_y, cv2.INTER_LINEAR)[145:, 195:377]
        cyl_ims.append(reim)
    for i,im in enumerate(w_ths):
        reim = cv2.remap(im, mat_x, mat_y, cv2.INTER_LINEAR)[145:, 195:377]
        cyl_ths.append(reim)
    
    # Stitch and blend the panoramas together
    N_ims = NUM_PICTURES

    s_ims = []
    s_ths = []
    s_vis = []
    
    
    for i in range(N_ims):
        im = np.zeros((cyl_ims[i].shape[0],cyl_ims[i].shape[1]+int(delta_col()*(N_ims-1)),cyl_ims[i].shape[2]),np.uint8)
        th = np.zeros(im.shape[:2], np.uint8)
        vi = np.zeros(im.shape, np.uint8)
        im[:cyl_ims[i].shape[0],int(delta_col()*i):int(delta_col()*i)+cyl_ims[i].shape[1]] = cyl_ims[i]
        th[:cyl_ims[i].shape[0],int(delta_col()*i):int(delta_col()*i)+cyl_ims[i].shape[1]] = cyl_ths[i]
        vi[:cyl_ims[i].shape[0],int(delta_col()*i):int(delta_col()*i)+cyl_ims[i].shape[1]] = cyl_vis[i]
        s_ims.append(im)
        s_ths.append(th)
        s_vis.append(vi)

    gpIms = []
    gpThs = []
    gpVis = []
    for i,im in enumerate(s_ims):
        G = im.copy()
        gpIms.append([G])
        Gth = s_ths[i].copy()
        gpThs.append([Gth])
        Gvi = s_vis[i].copy()
        gpVis.append([Gvi])
        for j in range(6):
            G = cv2.pyrDown(G)
            Gth = cv2.pyrDown(Gth)
            Gvi = cv2.pyrDown(Gvi)
            gpIms[i].append(G)
            gpThs[i].append(Gth)
            gpVis[i].append(Gvi)

    lpIms = []
    lpThs = []
    lpVis = []
    for i,gpIm in enumerate(gpIms):
        lpIms.append([gpIm[5]])
        lpThs.append([gpThs[i][5]])
        lpVis.append([gpVis[i][5]])
        for j in range(5,0,-1):
            GE = cv2.pyrUp(gpIm[j])
            GEth = cv2.pyrUp(gpThs[i][j])
            GEvi = cv2.pyrUp(gpVis[i][j])
            hmax = min(GE.shape[0], gpIm[j-1].shape[0])
            wmax = min(GE.shape[1], gpIm[j-1].shape[1])
            if i%2 == 0:
                L = cv2.subtract(gpIm[j-1][:hmax, :wmax], GE[:hmax, :wmax])
                Lth = cv2.subtract(gpThs[i][j-1][:hmax, :wmax], GEth[:hmax, :wmax])
                Lvi = cv2.subtract(gpVis[i][j-1][:hmax, :wmax], GEvi[:hmax, :wmax])
            else:
                L = cv2.subtract(gpIm[j-1][-hmax:, -wmax:], GE[-hmax:, -wmax:])
                Lth = cv2.subtract(gpThs[i][j-1][-hmax:, -wmax:], GEth[-hmax:, -wmax:])
                Lvi = cv2.subtract(gpVis[i][j-1][-hmax:, -wmax:], GEvi[-hmax:, -wmax:])
            lpIms[i].append(L)
            lpThs[i].append(Lth)
            lpVis[i].append(Lvi)
    
    LS = []
    for lx in zip(*lpIms):
        cols = lx[0].shape[1]
        part = lambda x: (x*cols)//N_ims
        ls = np.hstack((lx[i][:,part(i):part(i+1)] for i in range(N_ims)))
        LS.append(ls)
    
    TH = []
    for tx in zip(*lpThs):
        cols = tx[0].shape[1]
        part = lambda x: (x*cols)//N_ims
        ts = np.hstack((tx[i][:,part(i):part(i+1)] for i in range(N_ims)))
        TH.append(ts)
    
    VI = []
    for vx in zip(*lpVis):
        cols = vx[0].shape[1]
        part = lambda x: (x*cols)//N_ims
        vs = np.hstack((vx[i][:,part(i):part(i+1)] for i in range(N_ims)))
        VI.append(vs)
    
    ls_ = LS[0]
    ts_ = TH[0]
    vs_ = VI[0]
    for i in range(1,6):
        ls_ = cv2.pyrUp(ls_)
        ts_ = cv2.pyrUp(ts_)
        vs_ = cv2.pyrUp(vs_)
        hmax = min(ls_.shape[0], LS[i].shape[0])
        wmax = min(ls_.shape[1], LS[i].shape[1])
        ls_ = cv2.add(ls_[:hmax, :wmax], LS[i][:hmax, :wmax])
        ts_ = cv2.add(ts_[:hmax, :wmax], TH[i][:hmax, :wmax])
        vs_ = cv2.add(vs_[:hmax, :wmax], VI[i][:hmax, :wmax])
    
    cols = s_ims[0].shape[1]
    part = lambda x: (x*cols)//N_ims
    simpleOver = np.hstack((s_ims[i][:,part(i):part(i+1)] for i in range(N_ims)))
    simpleTemp = np.hstack((s_ths[i][:,part(i):part(i+1)] for i in range(N_ims)))
    simpleVisu = np.hstack((s_vis[i][:,part(i):part(i+1)] for i in range(N_ims)))

    # Create a temperature version of the IR Panorama
    temps = reconvertTemp(ts_, max_temp, min_temp)
    return (ls_, ts_, vs_, temps)

# Main Code

In [ ]:
count = 0

for dir in sorted(listdir(directory)):
 
    if count == 2:
        break
    count = count + 1
    dirname = directory + '/' +   dir + '/'

    if not os.path.exists(dirname + 'stitched'):
        os.makedirs(dirname + 'stitched')

    print(dirname )

    (simpleOver, simpleTemp, simpleVisu, temp) = process_set(dirname)

    cv2.imwrite(dirname + 'stitched/stitchedOverlay_0.png', simpleOver[0:550,:,:])
    cv2.imwrite(dirname + 'stitched/stitchedVisual_0.png', simpleVisu[0:550,:,:])
    #plt.imsave(dirname + 'stitched/stitchedtemps_0.jpg', simpleTemp, cmap = cm.thermal, vmin =65, vmax =85)

    np.savetxt(dirname + 'stitched/stitchedtemps_0.txt', temp,   delimiter=',')


/home/nilavro/data/Nilavra_z1/2017-12-20/1513736748855/1513746002780/
/home/nilavro/data/Nilavra_z1/2017-12-20/1513736748855/1513746303098/


# Shift Management

In [ ]:
def cross_image(im1, im2):
   # get rid of the color channels by performing a grayscale transform
   # the type cast into 'float' is to avoid overflows
    im1_gray = np.sum(im1.astype('float'), axis=2)
    im2_gray = np.sum(im2.astype('float'), axis=2)

   # get rid of the averages, otherwise the results are not good
    im1_gray -= np.mean(im1_gray)
    im2_gray -= np.mean(im2_gray)

   # calculate the correlation image; note the flipping of onw of the images
    return scipy.signal.fftconvolve(im1_gray, im2_gray[::-1,::-1], mode='same')

In [ ]:
count = 0
for dir in sorted(listdir(directory)):
    
    if (count == 0):
        pic1  = cv2.imread(directory + '/' + dir  + '/stitched/stitchedVisual_0.png')
    
    count = count + 1
    pic2  = cv2.imread(directory + '/' + dir  + '/stitched/stitchedVisual_0.png')

    corr_img = cross_image(pic1, pic2) 
    
    points = np.unravel_index(np.argmax(corr_img), corr_img.shape)
    shift = np.int(pic1.shape[1]/2) - points[1]

    if (shift > 1):
        rows, cols, channels = pic2.shape

        warp_matrix = np.float32([[1,0,-shift],[0,1,0]])
        new_pic2 = cv2.warpAffine(pic2, warp_matrix, (cols, rows)) 


        new_pic2[0:rows,(cols-shift):cols] = pic2[0:rows,0:shift]



        overlay  = cv2.imread(directory + '/' + dir  + '/stitched/stitchedOverlay_0.png')
        # temps  = cv2.imread(directory + '/' + dir  + '/stitched/stitchedtemps_0.png')
        tempstxt = np.loadtxt(directory + '/' + dir  +  '/stitched/stitchedtemps_0.txt', dtype = np.float64, delimiter=',')


        new_overlay =  cv2.warpAffine(overlay, warp_matrix, (cols, rows)) 
        new_overlay[0:rows,(cols-shift):cols] = overlay[0:rows,0:shift]


        # new_temps =  cv2.warpAffine(temps, warp_matrix, (cols, rows)) 
        # new_temps[0:rows,(cols-shift):cols] = temps[0:rows,0:shift]


        new_tempstxt = tempstxt



        new_tempstxt = np.roll(new_tempstxt, -shift) 


        cv2.imwrite(directory + '/' + dir  + '/stitched/stitchedVisual_0.png',new_pic2)
        cv2.imwrite(directory + '/' + dir  + '/stitched/stitchedOverlay_0.png',new_overlay)
        # cv2.imwrite(directory + '/' + dir  + '/stitched/stitchedtemps_0.png',new_temps)

        np.savetxt(directory + '/' + dir  + '/stitched/stitchedtemps_0.txt',new_tempstxt, delimiter=',') 

    print(dir, -shift)

# Plot

In [ ]:
pylab.rcParams['figure.figsize'] = 64, 48 
count = 0
for dir in sorted(listdir(directory)):
    print(dir, count)
    pic2  = cv2.imread(directory + '/' + dir  + '/stitched/stitchedVisual_0.png')
    # temp  = cv2.imread(directory + '/' + dir  + '/stitched/stitchedtemps_0.png')

    temps  = np.loadtxt(directory +  '/' + dir  +'/stitched/stitchedtemps_0.txt', dtype = np.float64, delimiter=',')
    f, axarr = plt.subplots(1,2) 
    axarr[0].imshow(pic2 )
    axarr[1].imshow(temps , cmap=cm.thermal ,vmin = 70, vmax = 90)
    # axarr[2].imshow(temp)
   
    count = count + 1
    major_ticks = np.arange(0, 3500, 50)                                              
    minor_ticks = np.arange(0, 600, 50)                                               

    # ax.set_xticks(major_ticks)                                                       
    # ax.set_xticks(minor_ticks, minor=True)                                           
    # ax.set_yticks(minor_ticks)                                                       
    # ax.set_yticks(minor_ticks, minor=True)                                               

    axarr[0].set_xticks(major_ticks)                                                       

    axarr[0].set_yticks(minor_ticks)                                                       


    axarr[0].grid(color='w', linestyle='-', linewidth=2, alpha= 50)    
    axarr[1].set_xticks(major_ticks)                                                       

    axarr[1].set_yticks(minor_ticks)                                                       


    axarr[1].grid(color='w', linestyle='-', linewidth=2, alpha= 50)     
    plt.show() 

# Read Energy

In [ ]:
'''
Strip prefix of directory  do this part later
'''

Dir_energy = '/home/nilavro/data/power/2017-12-19/1513711032463'

Energy = pd.read_csv(Dir_energy + 'Energy.csv',  sep=',', header='infer')

